**2. На Airbnb есть две основные группы пользователей:**

* Хозяева (хосты, hosts) – сдают жилье
* Путешественники (travelers) – снимают

**Начнем с анализа характеристик хозяев в таблице listings!**

**Пользователи, сдающие квартиры на Airbnb, зарегистрировались в разное время. Кто-то – очень давно, а кто-то совсем недавно. Давайте проверим, в какой месяц и год зарегистрировалось наибольшее количество новых хостов. В качестве ответа введите дату следующего формата: 2010-12**

* host_id – идентификатор хозяина
* host_since – дата регистрации как хост

Note: Сам хост может встретиться в таблице несколько раз, если сдает несколько помещений, поэтому не забудьте оставить уникальные значения host_id. Также обратите внимание на тип данных в host_since, возможно вам пригодится toStartOfMonth() для извлечения части даты и toDateOrNull() для обработки пустых значений.


Ответ: `2015-07`

```sql
SELECT 
    toStartOfMonth(toDateOrNull(host_since)) as start_of_month, 
    count(DISTINCT host_id) as NumHosts
FROM listings
GROUP BY start_of_month
ORDER BY NumHosts desc
limit 100
```

**3. Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t).**

**Значения частоты ответа хранятся как строки и включают значок %, который необходимо заменить на пустоту (''). После этого приведите столбец к нужному типу данных с помощью toInt32OrNull() и посчитайте среднюю частоту отклика в разбивке по тому, является ли хост суперхозяином или нет. В качестве ответа укажите наибольшее среднее.**

* host_response_rate – частота ответа
* host_is_superhost – является ли суперхозяином

Важный момент: у каждого host_id есть только одно уникальное значение частоты ответа, а также одна единая отметка суперхозяина. Чтобы посчитать показатели честно, нужно использовать подзапрос и взять уникальные комбинации с помощью DISTINCT.


Ответ: `98`

```sql
SELECT 
    host_is_superhost,
    avg(toInt32OrNull(replaceRegexpAll(host_response_rate, '%', ''))) as avg_rate, 
    min(toInt32OrNull(replaceRegexpAll(host_response_rate, '%', ''))) as min_rate,
    max(toInt32OrNull(replaceRegexpAll(host_response_rate, '%', ''))) as max_rate
FROM (
    select distinct host_id, host_is_superhost, host_response_rate
    from listings
    where host_is_superhost != ''
    )
GROUP BY host_is_superhost
order by avg_rate desc
```

**4. Сгруппируйте данные из listings по хозяевам (host_id) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает (у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. Таблицу отсортируйте по убыванию средней цены и убыванию host_id (в таком порядке). В качестве ответа укажите первый массив в результирующей таблице, состоящий более чем из двух id. Например:**

2343477,6340,32324

* host_id – идентификатор хозяина
* id – идентификатор жилья
* price – цена за ночь в конкретном месте


Ответ: `25757977,25759146,25802565,25802651,25802838,25802909,25803050,25803117,25803218,25803260`

```sql
SELECT 
    host_id,
    groupArray(id),
    groupArray(price),
    AVG(toFloat64OrNull(replaceRegexpAll(price, '[$, ]', ''))) AS AvgPricePerHost
FROM
    listings
GROUP BY host_id
HAVING length(groupArray(price)) > 1
order by AvgPricePerHost desc, host_id desc
limit 100
```

**5. Немного усложним предыдущую задачу, и посчитаем разницу между максимальной и минимальной установленной ценой у каждого хозяина. В качестве ответа укажите идентификатор хоста, у которого разница оказалась наибольшей.**

* host_id – идентификатор хозяина
* id – идентификатор жилья
* price – цена за ночь в конкретном месте


Ответ: `155140624`

```sql
SELECT 
    host_id,
    groupArray(id),
    max(toFloat64OrNull(replaceRegexpAll(price, '[$, ]', ''))) - min(toFloat64OrNull(replaceRegexpAll(price, '[$, ]', ''))) as price_diff
    
FROM
    listings
GROUP BY host_id
order by price_diff desc
limit 100
```

**6. Теперь сгруппируйте данные по типу жилья и выведите средние значения цены за ночь, размера депозита и цены уборки. Обратите внимание на тип данных, наличие значка $ и запятых в больших суммах. Для какого типа жилья среднее значение залога наибольшее?**

* room_type – тип сдаваемого жилья 
* price – цена за ночь
* security_deposit – залог за сохранность имущества
* cleaning_fee – плата за уборку



Ответ:
* **Entire home/apt**
* Private room
* Shared room


```sql
SELECT 
    room_type,
    avg(toFloat32OrNull(replaceRegexpAll(price, '[$,]',''))) as AvgPrice,
    avg(toFloat32OrNull(replaceRegexpAll(security_deposit, '[$,]',''))) as AvgSecDeposit,
    avg(toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]',''))) as AvgCleaningFee
FROM
    listings
Group by room_type
order by AvgSecDeposit desc
limit 100
```

**7. Отлично! В каких частях города средняя стоимость за ночь является наиболее низкой?**

**Сгруппируйте данные по neighbourhood_cleansed и посчитайте среднюю цену за ночь в каждом районе. В качестве ответа введите название места, где средняя стоимость за ночь ниже всего.**

* price – цена за ночь
* neighbourhood_cleansed – район/округ города


Ответ: `Neu-Hohenschönhausen Süd`

```sql
SELECT 
    neighbourhood_cleansed,
    avg(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as avg_price
FROM
    listings
Group by neighbourhood_cleansed
order by avg_price
limit 100
```

**8. В каких районах Берлина средняя площадь жилья, которое сдаётся целиком, является наибольшей? Отсортируйте по среднему и выберите топ-3.**

* neighbourhood_cleansed – район
* square_feet – площадь
* room_type – тип (нужный – 'Entire home/apt')


* Mitte
* **Schöneberg-Süd**
* Nord 1
* **Kaulsdorf**
* **Lichtenrade**
* Baumschulenweg


```sql
SELECT 
    neighbourhood_cleansed,
    avg(toFloat32OrNull(square_feet)) as avg_square_feet
FROM
    listings
WHERE room_type = 'Entire home/apt'
GROUP BY neighbourhood_cleansed
ORDER BY avg_square_feet desc
limit 3
```

**9. Напоследок давайте посмотрим, какая из представленных комнат расположена ближе всего к центру города. В качестве ответа укажите id объявления.**

* id – идентификатор жилья (объявления)
* room_type – тип жилья ('Private room')
* latitude – широта
* longitude – долгота
* 52.5200 с.ш., 13.4050 в.д – координаты центра Берлина


Ответ: `19765058`

```sql
SELECT 
    id,
    geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude)) as distance
FROM
    listings
WHERE room_type = 'Private room'
ORDER BY distance
limit 1
```